[View in Colaboratory](https://colab.research.google.com/github/ozanpekmezci/capstone/blob/master/Digit_Recognition.ipynb)

In [0]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
import numpy as np
from matplotlib import pyplot as plt
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
import cv2
import matplotlib.pyplot as plt
%matplotlib inline 

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# depth is 1 since greyscale
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
# Normalize data
X_train = X_train / 255
X_test = X_test / 255
# one-hot encoding
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
# number of classes which is 10 for our case
num_classes = y_test.shape[1]

11493376/11490434 [==============================] - 1s 0us/step


In [0]:
def deeper_cnn_model():
    model = Sequential()
    # 30 feature maps with size 5x5, mnist has the size 28x28
    model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu'))
    # MaxPooling2D layer will be our second layer where we will have a sample window of size 2 x 2
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(15, (3, 3), activation='relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Dropout(0.2))

    model.add(Flatten())

    model.add(Dense(128, activation='relu'))

    model.add(Dense(64, activation='relu'))
    
    model.add(Dense(32, activation='relu'))
    # output layer
    model.add(Dense(num_classes, activation='softmax', name='preds'))
    # configure training process
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [5]:
model = deeper_cnn_model()
model.summary()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)
scores = model.evaluate(X_test, y_test, verbose=0)
print("\nacc: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 24, 24)        780       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 12, 12)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 10, 10)        4065      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 5, 5)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 5, 5)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 375)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               48128     
__________

60000/60000 [==============================] - 6s 96us/step - loss: 0.3910 - acc: 0.8776 - val_loss: 0.0745 - val_acc: 0.9764
Epoch 2/10
60000/60000 [==============================] - 5s 75us/step - loss: 0.0925 - acc: 0.9714 - val_loss: 0.0486 - val_acc: 0.9846
Epoch 3/10
60000/60000 [==============================] - 5s 76us/step - loss: 0.0681 - acc: 0.9791 - val_loss: 0.0364 - val_acc: 0.9889
Epoch 4/10
60000/60000 [==============================] - 5s 77us/step - loss: 0.0542 - acc: 0.9835 - val_loss: 0.0340 - val_acc: 0.9887
Epoch 5/10
18600/60000 [========>.....................] - ETA: 3s - loss: 0.0453 - acc: 0.9863

60000/60000 [==============================] - 5s 76us/step - loss: 0.0468 - acc: 0.9852 - val_loss: 0.0328 - val_acc: 0.9894
Epoch 6/10
60000/60000 [==============================] - 5s 78us/step - loss: 0.0421 - acc: 0.9868 - val_loss: 0.0287 - val_acc: 0.9910
Epoch 7/10
60000/60000 [==============================] - 5s 80us/step - loss: 0.0386 - acc: 0.9876 - val_loss: 0.0273 - val_acc: 0.9914
Epoch 8/10
60000/60000 [==============================] - 5s 80us/step - loss: 0.0345 - acc: 0.9884 - val_loss: 0.0291 - val_acc: 0.9906
Epoch 9/10
12400/60000 [=====>........................] - ETA: 3s - loss: 0.0306 - acc: 0.9898

60000/60000 [==============================] - 5s 81us/step - loss: 0.0323 - acc: 0.9892 - val_loss: 0.0248 - val_acc: 0.9921
Epoch 10/10
60000/60000 [==============================] - 5s 81us/step - loss: 0.0291 - acc: 0.9904 - val_loss: 0.0260 - val_acc: 0.9916

acc: 99.16%


In [10]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving number-5-print-black.jpg to number-5-print-black.jpg
User uploaded file "number-5-print-black.jpg" with length 10346 bytes


In [11]:
img_pred = cv2.imread('number-5-print-black.jpg',  0)
# forces the image to have the input dimensions equal to those used in the training data (28x28)
if img_pred.shape != [28,28]:
    img2 = cv2.resize(img_pred,(28,28))
    img_pred = img2.reshape(28,28,-1);
else:
    img_pred = img_pred.reshape(28,28,-1);
    
# here also we inform the value for the depth = 1, number of rows and columns, which correspond 28x28 of the image.
img_pred = img_pred.reshape(1,1,28,28)
pred = model.predict_classes(img_pred)
pred_proba = model.predict_proba(img_pred)
pred_proba = "%.2f%%" % (pred_proba[0][pred]*100)
print(pred[0], "with prob of", pred_proba)

5 with prob of 100.00%
